## MNIST dataset을 이용한 이미지 분류 실습

### 5/19 실습

### 데이터 생성 및 전처리

In [ ]:
### 필요한 라이브러리 임폴트
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
### MNIST dataset 다운로드
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
### 데이터에 대한 확인
print(f'학습용 데이터의 자료형 : {type(X_train)}')
print('-'*80)
print(f'학습용 데이터의 모양 : {X_train.shape}')
print('-'*80)
print(f'학습용 레이블의 모양 : {y_train.shape}')
print('-'*80)
print(f'평가용 데이터의 자료형 : {type(X_test)}')
print(f'-'*80)
print(f'평가용 데이터의 모양 : {X_test.shape}')
print('-'*80)
print(f'평가용 레이블의 모양 : {y_test.shape}')

학습용 데이터의 자료형 : <class 'numpy.ndarray'>
--------------------------------------------------------------------------------
학습용 데이터의 모양 : (60000, 28, 28)
--------------------------------------------------------------------------------
학습용 레이블의 모양 : (60000,)
--------------------------------------------------------------------------------
평가용 데이터의 자료형 : <class 'numpy.ndarray'>
--------------------------------------------------------------------------------
평가용 데이터의 모양 : (10000, 28, 28)
--------------------------------------------------------------------------------
평가용 레이블의 모양 : (10000,)


In [ ]:
### 손글씨 데이터의 모양(차원) 변경 : 2차원 --> 3차원
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 결과 확인하기
print(f'전처리 후 학습용 데이터의 모양 = {X_train.shape}')
print('-'*80)
print(f'전처리 후 평가용 데이터의 모양 = {X_test.shape}')

전처리 후 학습용 데이터의 모양 = (60000, 28, 28, 1)
--------------------------------------------------------------------------------
전처리 후 평가용 데이터의 모양 = (10000, 28, 28, 1)


In [ ]:
### MinMaxScaling 실행 --> normalization
X_train = X_train/255
X_test = X_test/255

# 결과 확인
print(f'학습용 데이터의 픽셀의 최대값 = {np.max(X_train)}, 최소값 : {np.min(X_train)}')
print('-'*80)
print(f'평가용 데이터의 픽셀의 최대값 : {np.max(X_test)}, 최소값 : {np.min(X_test)}')

학습용 데이터의 픽셀의 최대값 = 1.0, 최소값 : 0.0
--------------------------------------------------------------------------------
평가용 데이터의 픽셀의 최대값 : 1.0, 최소값 : 0.0


In [ ]:
### 학습용 데이터 --> 검증용으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=99)

In [ ]:
### 데이터 분할의 결과 확인
print(f'학습용 데이터의 모양 : {X_train.shape}')
print('-'*80)
print(f'검증용 데이터의 모양 : {X_val.shape}')

학습용 데이터의 모양 : (48000, 28, 28, 1)
--------------------------------------------------------------------------------
검증용 데이터의 모양 : (12000, 28, 28, 1)


### CNN 모델을 이용한 이미지 분류

In [ ]:
### 구글 서버와 내 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 작업 디렉토러 변경하기
%cd '/content/drive/MyDrive/KDT/비정형텍스트분석'

/content/drive/MyDrive/KDT/비정형텍스트분석


In [ ]:
### ### 랜덤 시드 설정
tf.random.set_seed(99)

In [ ]:
### 모델의 구조 생성 함수 정의

def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    return model

In [ ]:
### 모델의 구조 확인 --> model.summary()
model = create_model()
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 10)                3

In [ ]:
### 2번째 Conv2D layer --> 가중치의 수 계산

# feature map 32 --> filter 1set
num_weights = (3*3*32)
print(num_weights)

print('-'*80)

# filter = 64(set)
num_weights_filters = (3*3*32)*64 + 64
print(num_weights_filters)

288
--------------------------------------------------------------------------------
18496


In [ ]:
### 모델 컴퍼일하기 --> 손실 함수 + 최적화 함수 + 검증용 데이터에 대한 평가 척도 --> 모델 완성
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
### 학습 진행하기

# ModelCheckpoint() 객체 생성 --> 매개 변수 설정하기
file_path = 'cnn_mnist.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=file_path,
    monitor='val_loss',
    save_best_only=True
)

model.fit(x=X_train,
          y=y_train,
          batch_size=512,
          epochs=10,
          validation_data=(X_val, y_val),
          callbacks=[checkpoint])

Epoch 1/10
94/94 [==============================] - 12s 17ms/step - loss: 0.6563 - accuracy: 0.8329 - val_loss: 0.2340 - val_accuracy: 0.9302
Epoch 2/10
94/94 [==============================] - 1s 11ms/step - loss: 0.1610 - accuracy: 0.9534 - val_loss: 0.1150 - val_accuracy: 0.9675
Epoch 3/10
94/94 [==============================] - 1s 13ms/step - loss: 0.0932 - accuracy: 0.9731 - val_loss: 0.0845 - val_accuracy: 0.9768
Epoch 4/10
94/94 [==============================] - 1s 14ms/step - loss: 0.0694 - accuracy: 0.9793 - val_loss: 0.0771 - val_accuracy: 0.9794
Epoch 5/10
94/94 [==============================] - 1s 12ms/step - loss: 0.0585 - accuracy: 0.9822 - val_loss: 0.0656 - val_accuracy: 0.9819
Epoch 6/10
94/94 [==============================] - 1s 12ms/step - loss: 0.0527 - accuracy: 0.9840 - val_loss: 0.0620 - val_accuracy: 0.9815
Epoch 7/10
94/94 [==============================] - 1s 12ms/step - loss: 0.0448 - accuracy: 0.9861 - val_loss: 0.0554 - val_accuracy: 0.9845
Epoch 8/10
9